In [ ]:


import os, cv2, json, math, random, torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from transformers import AutoImageProcessor, AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BASE_DIR  = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
AUTH_DIR  = f"{BASE_DIR}/train_images/authentic"
FORG_DIR  = f"{BASE_DIR}/train_images/forged"
MASK_DIR  = f"{BASE_DIR}/train_masks"
TEST_DIR  = f"{BASE_DIR}/test_images"
DINO_PATH = "/kaggle/input/dinov2/pytorch/base/1"

IMG_SIZE = 256
BATCH_SIZE = 1
EPOCHS_SEG = 1
LR_SEG = 3e-4
WEIGHT_DECAY = 1e-4


class ForgerySegDataset(Dataset):
    def __init__(self, auth_paths, forg_paths, mask_dir, img_size=256):
        self.samples = []
        for p in forg_paths:
            m = os.path.join(mask_dir, Path(p).stem + ".npy")
            if os.path.exists(m):
                self.samples.append((p, m))
        for p in auth_paths:
            self.samples.append((p, None))
        self.img_size = img_size
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        img_path, mask_path = self.samples[idx]
        img = Image.open(img_path).convert("RGB")
        w, h = img.size
        if mask_path is None:
            mask = np.zeros((h, w), np.uint8)
        else:
            m = np.load(mask_path)
            if m.ndim == 3: m = np.max(m, axis=0)
            mask = (m > 0).astype(np.uint8)
        img_r = img.resize((IMG_SIZE, IMG_SIZE))
        mask_r = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
        img_t = torch.from_numpy(np.array(img_r, np.float32)/255.).permute(2,0,1)
        mask_t = torch.from_numpy(mask_r[None, ...].astype(np.float32))
        return img_t, mask_t

# ----------------------------
# 🧠 MODEL (DINOv2 + Decoder)
# ----------------------------
from transformers import AutoImageProcessor, AutoModel
processor = AutoImageProcessor.from_pretrained(DINO_PATH, local_files_only=True)
encoder = AutoModel.from_pretrained(DINO_PATH, local_files_only=True).eval().to(device)

class DinoTinyDecoder(nn.Module):
    def __init__(self, in_ch=768, out_ch=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch,256,3,padding=1), nn.ReLU(),
            nn.Conv2d(256,64,3,padding=1), nn.ReLU(),
            nn.Conv2d(64,out_ch,1)
        )
    def forward(self, f, size):
        return self.net(F.interpolate(f, size=size, mode="bilinear", align_corners=False))

class DinoSegmenter(nn.Module):
    def __init__(self, encoder, processor):
        super().__init__()
        self.encoder, self.processor = encoder, processor
        for p in self.encoder.parameters(): p.requires_grad = False
        self.seg_head = DinoTinyDecoder(768,1)
    def forward_features(self,x):
        imgs = (x*255).clamp(0,255).byte().permute(0,2,3,1).cpu().numpy()
        inputs = self.processor(images=list(imgs), return_tensors="pt").to(x.device)
        with torch.no_grad(): feats = self.encoder(**inputs).last_hidden_state
        B,N,C = feats.shape
        fmap = feats[:,1:,:].permute(0,2,1)
        s = int(math.sqrt(N-1))
        fmap = fmap.reshape(B,C,s,s)
        return fmap
    def forward_seg(self,x):
        fmap = self.forward_features(x)
        return self.seg_head(fmap,(IMG_SIZE,IMG_SIZE))

# ----------------------------
# 🚀 TRAINING
# ----------------------------
auth_imgs = sorted([str(Path(AUTH_DIR)/f) for f in os.listdir(AUTH_DIR)])
forg_imgs = sorted([str(Path(FORG_DIR)/f) for f in os.listdir(FORG_DIR)])
train_auth, val_auth = train_test_split(auth_imgs, test_size=0.2, random_state=42)
train_forg, val_forg = train_test_split(forg_imgs, test_size=0.2, random_state=42)

train_loader = DataLoader(ForgerySegDataset(train_auth, train_forg, MASK_DIR),
                          batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

model_seg = DinoSegmenter(encoder, processor).to(device)
opt_seg = optim.AdamW(model_seg.seg_head.parameters(), lr=LR_SEG, weight_decay=WEIGHT_DECAY)
crit_seg = nn.BCEWithLogitsLoss()

for e in range(EPOCHS_SEG):
    model_seg.train()
    total_loss = 0
    for x,m in tqdm(train_loader, desc=f"[Segmentation] Epoch {e+1}/{EPOCHS_SEG}"):
        x,m = x.to(device),m.to(device)
        loss = crit_seg(model_seg.forward_seg(x),m)
        opt_seg.zero_grad(); loss.backward(); opt_seg.step()
        total_loss += loss.item()
    print(f"  → avg_loss={total_loss/len(train_loader):.4f}")
torch.save(model_seg.state_dict(),"model_seg_final.pt")

# ----------------------------
# 🧠 INFERENCE UTILS
# ----------------------------
@torch.no_grad()
def segment_prob_map(pil):
    x = torch.from_numpy(np.array(pil.resize((IMG_SIZE, IMG_SIZE)), np.float32)/255.).permute(2,0,1)[None].to(device)
    prob = torch.sigmoid(model_seg.forward_seg(x))[0,0].cpu().numpy()
    return prob

def enhanced_adaptive_mask(prob, alpha_grad=0.35):
    gx = cv2.Sobel(prob, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(prob, cv2.CV_32F, 0, 1, ksize=3)
    grad_mag = np.sqrt(gx**2 + gy**2)
    grad_norm = grad_mag / (grad_mag.max() + 1e-6)
    enhanced = (1 - alpha_grad) * prob + alpha_grad * grad_norm
    enhanced = cv2.GaussianBlur(enhanced, (3,3), 0)
    thr = np.mean(enhanced) + 0.3 * np.std(enhanced)
    mask = (enhanced > thr).astype(np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    return mask, thr

def finalize_mask(prob, orig_size):
    mask, thr = enhanced_adaptive_mask(prob)
    mask = cv2.resize(mask, orig_size, interpolation=cv2.INTER_NEAREST)
    return mask, thr

def pipeline_final(pil):
    prob = segment_prob_map(pil)
    mask, thr = finalize_mask(prob, pil.size)
    area = int(mask.sum())
    mean_inside = float(prob[cv2.resize(mask,(IMG_SIZE,IMG_SIZE),interpolation=cv2.INTER_NEAREST)==1].mean()) if area>0 else 0.0
    # 🔹 condition de filtrage
    if area < 400 or mean_inside < 0.35:
        return "authentic", None, {"area": area, "mean_inside": mean_inside, "thr": thr}
    return "forged", mask, {"area": area, "mean_inside": mean_inside, "thr": thr}


from sklearn.metrics import f1_score
val_items = [(p, 1) for p in val_forg[:10]]
results = []
for p,_ in tqdm(val_items, desc="Validation forged-only"):
    pil = Image.open(p).convert("RGB")
    label, m_pred, dbg = pipeline_final(pil)
    m_gt = np.load(Path(MASK_DIR)/f"{Path(p).stem}.npy")
    if m_gt.ndim==3: m_gt=np.max(m_gt,axis=0)
    m_gt=(m_gt>0).astype(np.uint8)
    m_pred=(m_pred>0).astype(np.uint8) if m_pred is not None else np.zeros_like(m_gt)
    f1 = f1_score(m_gt.flatten(), m_pred.flatten(), zero_division=0)
    results.append((Path(p).stem, f1, dbg))
print("\n📊 F1-score par image falsifiée:\n")
for cid,f1,dbg in results:
    print(f"{cid} — F1={f1:.4f} | area={dbg['area']} mean={dbg['mean_inside']:.3f} thr={dbg['thr']:.3f}")
print(f"\n🌟 Moyenne F1 (falsifiées) = {np.mean([r[1] for r in results]):.4f}")



In [ ]:
import os, cv2, json, math, random, torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from transformers import AutoImageProcessor, AutoModel
import albumentations as A
from albumentations.pytorch import ToTensorV2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Example directories for a generic image segmentation task (e.g., medical imaging)
# Replace with your actual dataset paths
BASE_DIR = "/path/to/your/dataset" 
IMG_DIR = f"{BASE_DIR}/images"
MASK_DIR = f"{BASE_DIR}/masks"
IMG_SIZE = 256
BATCH_SIZE = 8
EPOCHS_SEG = 1
LR_SEG = 3e-4
WEIGHT_DECAY = 1e-4

# --- Augmentations (Using standard Albumentations) ---
def get_train_transforms(img_size):
    return A.Compose([
        A.RandomResizedCrop(height=img_size, width=img_size, scale=(0.8, 1.0), ratio=(0.9, 1.1), p=0.6),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.2),
        A.Resize(img_size, img_size), # Final resize ensures consistent output size
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

def get_valid_transforms(img_size):
    return A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

# --- Dataset Class for Generic Segmentation ---
class GenericSegDataset(Dataset):
    def __init__(self, samples, transforms=None):
        # samples is a list of tuples (img_path, mask_path)
        self.samples = samples 
        self.transforms = transforms

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        img_path, mask_path = self.samples[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) # Assuming single channel mask

        if self.transforms:
            augmented = self.transforms(image=img, mask=mask)
            img_t = augmented['image']
            mask_t = augmented['mask'].unsqueeze(0) # Add channel dimension
        else:
            # Fallback (shouldn't be used with this setup)
            img_t = torch.from_numpy(np.array(img, np.float32)/255.).permute(2,0,1)
            mask_t = torch.from_numpy(mask[None, ...].astype(np.float32))

        # Ensure mask is binary (0 or 1)
        mask_t = (mask_t > 0).float() 

        return img_t, mask_t

# ----------------------------
# 🧠 MODEL (Example U-Net like Decoder)
# ----------------------------
# This is a simplified decoder example. Replace with a proper segmentation model architecture.
class SimpleDecoder(nn.Module):
    def __init__(self, in_ch=768, out_ch=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch,256,3,padding=1), nn.ReLU(),
            nn.Conv2d(256,64,3,padding=1), nn.ReLU(),
            nn.Conv2d(64,out_ch,1)
        )
    
    def forward(self, f, size):
        x = self.net(f)
        # Upsample using interpolation to match desired spatial 'size' (H, W)
        x = F.interpolate(x, size=size, mode='bilinear', align_corners=False)
        return x

# Example: Using a pre-trained encoder (like a generic vision transformer) and a decoder
# Replace with an appropriate encoder for your task.
# This DINO path is for illustration; you'd use a general purpose pre-trained model.
# processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224") 
# encoder = AutoModel.from_pretrained("google/vit-base-patch16-224").eval().to(device)

# ----------------------------
# Loss Function (Added standard combined loss)
# ----------------------------
def dice_loss(pred, target):
    smooth = 1e-5
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum()
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1. - dice

class CombinedLoss(nn.Module):
    def __init__(self, bce_weight=0.5, dice_weight=0.5):
        super().__init__()
        # Use BCEWithLogitsLoss for numerical stability
        self.bce = nn.BCEWithLogitsLoss()
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight

    def forward(self, pred, target):
        bce_loss = self.bce(pred, target)
        # Apply sigmoid before dice calculation for dice loss
        dice = dice_loss(torch.sigmoid(pred), target) 
        return self.bce_weight * bce_loss + self.dice_weight * dice

# ----------------------------
# DATA SETUP (Boilerplate)
# ----------------------------

# ... (You would need code here to generate the `all_samples` list 
#      by pairing image paths from IMG_DIR with corresponding mask paths from MASK_DIR
#      and then split it into `train_samples` and `valid_samples` using 
#      sklearn's train_test_split) ...

# Example of how you would instantiate the dataset with transforms:
# # Assume all_samples is a list of (image_path, mask_path) tuples
# train_samples, valid_samples = train_test_split(all_samples, test_size=0.2, random_state=42)
# train_dataset = GenericSegDataset(train_samples, transforms=get_train_transforms(IMG_SIZE))
# valid_dataset = GenericSegDataset(valid_samples, transforms=get_valid_transforms(IMG_SIZE))

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
# valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Example of model instantiation (assuming a generic encoder outputting features)
# Replace with actual encoder output channel size
# decoder = SimpleDecoder(in_ch=768, out_ch=1).to(device)
# loss_fn = CombinedLoss().to(device)
# optimizer = optim.AdamW(decoder.parameters(), lr=LR_SEG, weight_decay=WEIGHT_DECAY)

# ... (Training and validation loop would follow here) ...


In [ ]:
import os, json, cv2
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

# --- RLE encodeur Kaggle ---
def rle_encode(mask: np.ndarray, fg_val: int = 1) -> str:
    pixels = mask.T.flatten()
    dots = np.where(pixels == fg_val)[0]
    if len(dots) == 0:
        return "authentic"
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return json.dumps([int(x) for x in run_lengths])

# --- Paths ---
TEST_DIR = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/test_images"
SAMPLE_SUB = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/sample_submission.csv"
OUT_PATH = "submission.csv"

# --- Inference ---
rows = []
for f in tqdm(sorted(os.listdir(TEST_DIR)), desc="Inference on Test Set"):
    pil = Image.open(Path(TEST_DIR)/f).convert("RGB")
    label, mask, dbg = pipeline_final(pil)     # ta fonction de décision actuelle

    # Force format masque
    if mask is not None:
        mask = np.array(mask, dtype=np.uint8)
    else:
        mask = np.zeros(pil.size[::-1], np.uint8)

    if label == "authentic":
        annot = "authentic"
    else:
        annot = rle_encode((mask > 0).astype(np.uint8))

    rows.append({
        "case_id": Path(f).stem,
        "annotation": annot,
        "area": int(dbg.get("area", mask.sum())),
        "mean": float(dbg.get("mean_inside", 0.0)),
        "thr": float(dbg.get("thr", 0.0))
    })

sub = pd.DataFrame(rows)
ss = pd.read_csv(SAMPLE_SUB)
ss["case_id"] = ss["case_id"].astype(str)
sub["case_id"] = sub["case_id"].astype(str)
final = ss[["case_id"]].merge(sub, on="case_id", how="left")
final["annotation"] = final["annotation"].fillna("authentic")
final[["case_id", "annotation"]].to_csv(OUT_PATH, index=False)

print(f"\n Saved submission file: {OUT_PATH}")
print(final.head(10))

# --- Visualisation (seulement si forged) ---
sample_files = sorted(os.listdir(TEST_DIR))[:5]
for f in sample_files:
    pil = Image.open(Path(TEST_DIR)/f).convert("RGB")
    label, mask, dbg = pipeline_final(pil)

    mask = np.array(mask, dtype=np.uint8) if mask is not None else np.zeros(pil.size[::-1], np.uint8)
    print(f"{'🔴' if label=='forged' else '🟢'} {f}: {label} | area={mask.sum()} mean={dbg.get('mean_inside', 0):.3f}")

    if label == "authentic":
        plt.figure(figsize=(5,5))
        plt.imshow(pil)
        plt.title(f"{f} — Authentic")
        plt.axis("off")
        plt.show()
    else:
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1); plt.imshow(pil); plt.title("Original"); plt.axis("off")
        plt.subplot(1,2,2); plt.imshow(pil); plt.imshow(mask, alpha=0.45, cmap="Blues"); 
        plt.title("Predicted Mask"); plt.axis("off")
        plt.show()
